In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
plt.rc('font', family='Malgun Gothic') 
plt.rc('axes', unicode_minus=False)
import matplotlib.font_manager as fm
font_path = 'C:\Windows\Fonts\HMFMPYUN.TTF'
font_prop = fm.FontProperties(fname=font_path, size=20)
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout,Embedding, SimpleRNN, Flatten, Conv1D, GlobalMaxPooling1D, MaxPooling1D
from tensorflow.keras import models
import seaborn as sns
import tensorflow.keras.utils as utils
from tensorflow.keras.callbacks import Callback

In [2]:
suwon=pd.read_csv('data/suwon_dnn.csv', encoding='cp949', index_col=0)

In [3]:
suwon=suwon[['요일', '노면상태', '기상상태', '도로형태', '가해운전자_차종', '가해운전자_성별',
       '가해운전자_연령', '행정동', '발생월', '발생시간','사고내용']]

In [4]:
suwon=suwon[suwon['행정동']!='수원시']

In [5]:
suwon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23774 entries, 1 to 24359
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   요일        23774 non-null  object
 1   노면상태      23774 non-null  object
 2   기상상태      23774 non-null  object
 3   도로형태      23774 non-null  object
 4   가해운전자_차종  23774 non-null  object
 5   가해운전자_성별  23774 non-null  object
 6   가해운전자_연령  23774 non-null  int64 
 7   행정동       23774 non-null  object
 8   발생월       23774 non-null  int64 
 9   발생시간      23774 non-null  int64 
 10  사고내용      23774 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 2.2+ MB


In [6]:
suwon.isna().sum()

요일          0
노면상태        0
기상상태        0
도로형태        0
가해운전자_차종    0
가해운전자_성별    0
가해운전자_연령    0
행정동         0
발생월         0
발생시간        0
사고내용        0
dtype: int64

In [7]:
suwon.columns

Index(['요일', '노면상태', '기상상태', '도로형태', '가해운전자_차종', '가해운전자_성별', '가해운전자_연령', '행정동',
       '발생월', '발생시간', '사고내용'],
      dtype='object')

In [8]:
yoil2idx={'일요일':6 , '월요일':0, '화요일':1, '수요일':2, '목요일':3, '금요일':4, '토요일':5}

In [9]:
nom2idx={'포장 - 건조':7, '포장 - 습기':8, '포장 - 결빙':9, '포장 - 적설':10, '포장 - 기타':11, '비포장 - 습기':12, '비포장 - 건조':13, '포장 - 젖음/습기':14, '포장 - 서리/결빙':15, '비포장 - 젖음/습기':16, '포장 - 해빙':17, '포장 - 침수':18}

In [10]:
gs2idx={'맑음':19, '흐림':20, '안개':21, '눈':22, '비':23, '기타':24}

In [11]:
doro2idx={'단일로 - 기타':25, '교차로 - 교차로부근':26, '교차로 - 교차로안':27, '단일로 - 횡단보도상':28, '단일로 - 횡단보도부근':29, '미분류 - 미분류':30, '기타 - 기타':31, '단일로 - 터널':32, '단일로 - 교량':33, '교차로 - 교차로횡단보도내':34, '단일로 - 지하차도(도로)내':35, '단일로 - 고가도로위':36, '주차장 - 주차장':37}

In [12]:
gacha2idx={'승용':38, '화물':39, '승합':40, '이륜':41, '원동기':42, '자전거':43, '건설기계':44, '특수':45, '농기계':46, '기타불명':47, '사륜오토바이(ATV)':48, '개인형이동수단(PM)':49}

In [13]:
gasung2idx={'남':50, '여':51}

In [14]:
suwon[suwon.columns[6]].unique() # +52

array([42, 57, 26, 23, 31, 36, 48, 28, 54, 47, 49, 32, 27, 29, 50, 33, 55,
       22, 41, 60, 46, 64, 43, 58, 72, 12, 53, 51, 56, 69, 24, 63, 19, 74,
       71, 59, 45, 77, 35, 82, 52, 62, 61, 30, 39, 18, 44, 17, 40, 25, 21,
       20, 37, 16, 38, 73, 66, 67, 68, 76, 70, 34, 13, 80, 65,  8, 75, 79,
       78, 14, 15, 11, 81,  9, 10,  6, 83, 85,  7, 84, 86, 88,  4, 87, 89,
       96], dtype=int64)

In [15]:
dong2idx={'대황교동':149, '인계동':150, '우만동':151, '영통동':152, '세류동':153, '파장동':154, '권선동':155, '송죽동':156, '영화동':157,
       '정자동':158, '곡반정동':159, '이의동':160, '화서동':161, '고색동':162, '서둔동':163, '입북동':164, '원천동':165, '조원동':166,
       '연무동':167, '매탄동':168, '이목동':169, '망포동':170, '평동':171, '매교동':172, '탑동':173, '신동':174, '매산로1가':175,
       '율전동':176, '금곡동':177, '하동':178, '천천동':179, '북수동':180, '팔달로3가':181, '지동':182, '중동':183, '남수동':184,
       '하광교동':185, '당수동':186, '구운동':187, '고등동':188, '팔달로1가':189, '영동':190, '매산로2가':191, '교동':192, '팔달로2가':193,
       '매향동':194, '오목천동':195, '매산로3가':196, '호매실동':197, '구천동':198, '신풍동':199, '상광교동':200, '평리동':201, '장안동':202,
       '장지동':203, '남창동':204}

In [16]:
suwon[suwon.columns[8]].unique() # +204

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [17]:
suwon[suwon.columns[9]].unique() # +217

array([ 4,  5,  6, 11, 12, 15, 18,  1,  9, 16, 19, 22, 23,  3, 13, 17, 20,
        7,  8, 10, 21,  0, 14,  2], dtype=int64)

In [18]:
suwon_cp=suwon.copy()

In [19]:
suwon_cp['요일']=suwon_cp['요일'].map(yoil2idx)

In [20]:
#suwon_cp['요일']=(suwon_cp['요일']*(len(yoil2idx)-1)).map(idx2yoil)

In [21]:
suwon_cp['노면상태']=suwon_cp['노면상태'].map(nom2idx)

In [22]:
suwon_cp['기상상태']=suwon_cp['기상상태'].map(gs2idx)

In [23]:
suwon_cp['도로형태']=suwon_cp['도로형태'].map(doro2idx)

In [24]:
suwon_cp['가해운전자_차종']=suwon_cp['가해운전자_차종'].map(gacha2idx)

In [25]:
suwon_cp['가해운전자_성별']=suwon_cp['가해운전자_성별'].map(gasung2idx)

In [26]:
suwon_cp['가해운전자_연령']=suwon_cp['가해운전자_연령']+52

In [27]:
suwon_cp['행정동']=suwon_cp['행정동'].map(dong2idx)

In [28]:
suwon_cp['발생월']=suwon_cp['발생월']+204

In [29]:
suwon_cp['발생시간']=suwon_cp['발생시간']+217

In [30]:
suwon.columns

Index(['요일', '노면상태', '기상상태', '도로형태', '가해운전자_차종', '가해운전자_성별', '가해운전자_연령', '행정동',
       '발생월', '발생시간', '사고내용'],
      dtype='object')

In [31]:
dataset=suwon_cp.to_numpy()

In [32]:
dataset

array([[  4,   7,  19, ..., 205, 221,   1],
       [  4,   7,  19, ..., 205, 221,   1],
       [  4,   7,  19, ..., 205, 222,   0],
       ...,
       [  3,   7,  19, ..., 216, 235,   0],
       [  3,   7,  19, ..., 216, 236,   0],
       [  3,   7,  19, ..., 216, 236,   0]], dtype=int64)

In [33]:
vocab_size=dataset.max()+1

In [34]:
X = dataset[:,:-1]
Y = dataset[:,-1]


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((19019, 10), (19019,), (4755, 10), (4755,))

In [ ]:
model = Sequential()


model.add(Embedding(input_dim=vocab_size, output_dim=vocab_size-1, input_length=X.shape[1]))
model.add(SimpleRNN(256))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist=model.fit(X_train, Y_train, epochs=50, batch_size=50, verbose=2, validation_split=0.2)

In [ ]:
model = Sequential()


model.add(Embedding(input_dim=vocab_size, output_dim=vocab_size-1, input_length=X.shape[1]))
model.add(SimpleRNN(256))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

class LossHistory(Callback):
    def __init__(self):
        self.epoch = 0
        self.losses = []
        self.accuracy=[]
        self.val_loss = []
        self.val_accuracy = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        self.val_loss.append(logs.get('val_loss'))
        self.val_accuracy.append(logs.get('val_accuracy'))
        if self.epoch % 100 == 0:
            print("epoch: {0} - loss: {1:8.6f} - acc: {2:8.6f} - val_loss: {3:8.6f} - val_acc: {4:8.6f}".format(self.epoch, logs.get('loss'), logs.get('accuracy'), logs.get('val_loss'), logs.get('val_accuracy')))
        self.epoch += 1
history = LossHistory()
for epoch in range(2000):
    model.fit(X_train, Y_train, epochs=1, batch_size=1, verbose=0, shuffle=False, validation_data=(X_val,Y_val),
             callbacks=[history])
    if (history.losses[-1] < 1e-5):
        print("epoch:{} - loss:{} - 종료".\
              format(epoch, history.losses[-1]))
        break;
    model.reset_states() # 훈련할 때마다 이전 데이터와 상관없이 다른 데이터라고 판단

In [37]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=vocab_size-1, input_length=X.shape[1]))
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist=model.fit(X_train, Y_train, epochs=50, batch_size=50, verbose=2, validation_split=0.2)

Epoch 1/50
305/305 - 4s - loss: 0.5670 - accuracy: 0.7474 - val_loss: 0.5600 - val_accuracy: 0.7489
Epoch 2/50
305/305 - 2s - loss: 0.5597 - accuracy: 0.7476 - val_loss: 0.5635 - val_accuracy: 0.7487
Epoch 3/50
305/305 - 2s - loss: 0.5543 - accuracy: 0.7475 - val_loss: 0.5667 - val_accuracy: 0.7487
Epoch 4/50
305/305 - 2s - loss: 0.5482 - accuracy: 0.7476 - val_loss: 0.5707 - val_accuracy: 0.7479
Epoch 5/50
305/305 - 2s - loss: 0.5383 - accuracy: 0.7494 - val_loss: 0.5772 - val_accuracy: 0.7458
Epoch 6/50
305/305 - 2s - loss: 0.5260 - accuracy: 0.7537 - val_loss: 0.5858 - val_accuracy: 0.7440
Epoch 7/50
305/305 - 2s - loss: 0.5116 - accuracy: 0.7585 - val_loss: 0.5924 - val_accuracy: 0.7345
Epoch 8/50
305/305 - 3s - loss: 0.4938 - accuracy: 0.7663 - val_loss: 0.6230 - val_accuracy: 0.6903
Epoch 9/50
305/305 - 2s - loss: 0.4727 - accuracy: 0.7794 - val_loss: 0.6479 - val_accuracy: 0.6998
Epoch 10/50
305/305 - 2s - loss: 0.4514 - accuracy: 0.7894 - val_loss: 0.6726 - val_accuracy: 0.6853

In [66]:
pred=model.predict(X_train)

In [70]:
Y_test

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [75]:
((pred>0.5).reshape(19019)==Y_train).sum()/19019

0.9226562910773437

In [ ]:
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=64)
print('## evaluation loss and_metrics ##')
print(loss_and_metrics)